In [1]:
import os
import numpy as np
import random

import torch
import torch.nn as nn

from model import *

import arguments

In [2]:
import utils.load_dataset
import utils.data_loader
import utils.metrics
from utils.early_stop import EarlyStopping, Stop_args

In [3]:
from utils.metrics import auc

In [4]:
dataset = 'yahooR3'
base_model_args = {'emb_dim': 10, 'learning_rate': 0.01, 'imputaion_lambda': 0.01, 'weight_decay': 1}
weight1_model_args ={'learning_rate': 0.1, 'weight_decay': 0.001}
weight2_model_args =  {'learning_rate': 1e-3, 'weight_decay': 1e-2}
imputation_model_args = {'learning_rate': 1e-1, 'weight_decay': 1e-4}
training_args =  {'batch_size': 1024, 'epochs': 100, 'patience': 20, 'block_batch': [1000, 100]}
uniform_ratio = 0.05
seed = 0

In [5]:
from train_implicit import setup_seed
setup_seed(seed)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


cuda


## Load Dataset

In [7]:
dataset='yahooR3'
training_args = {'batch_size': 1024,
                 'epochs': 500, 
                 'patience': 60, 
                 'block_batch': [6000, 500]}

base_model_args = {'emb_dim': 10, 
                   'learning_rate': 0.0001, 
                   'imputaion_lambda': 1, 
                   'weight_decay': 1}
weight1_model_args = {'learning_rate': 0.01, 'weight_decay': 0.01}
weight2_model_args = {'learning_rate': 1e-3, 'weight_decay': 1e-2}
imputation_model_args = {'emb_dim': 10, 'learning_rate': 1e-1, 'weight_decay': 1e-4}

In [8]:
# args = arguments.parse_args()
# para(args)
setup_seed(seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [9]:
# To generate implicit feedback data, we also use the aforementioned
# datasets, but abandon the negative feedback of the training data.
train, unif_train, validation, test = utils.load_dataset.load_dataset(data_name=dataset, type = 'implicit', seed = seed, device=device)


In [10]:
train.shape, unif_train.shape, validation.shape, test.shape

(torch.Size([15400, 1000]),
 torch.Size([15400, 1000]),
 torch.Size([15400, 1000]),
 torch.Size([15400, 1000]))

In [11]:
train

tensor(indices=tensor([[    0,     0,     0,  ..., 15399, 15399, 15399],
                       [   13,   152,   169,  ...,   563,   636,   948]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       device='cuda:0', size=(15400, 1000), nnz=125077, layout=torch.sparse_coo)

In [12]:
train._indices().shape, train._values().shape, train._nnz()

(torch.Size([2, 125077]), torch.Size([125077]), 125077)

In [13]:
from train_implicit import train_and_eval

# train_and_eval(train, 
#                unif_train, 
#                validation, 
#                test,
#                device,
#                base_model_args = base_model_args, 
#                weight1_model_args = weight1_model_args,
#                weight2_model_args = weight2_model_args,
#                imputation_model_args = imputation_model_args, 
#                training_args = training_args)

## Proposed Model - AutoDebias_RD

In [14]:
train_data, unif_train_data, val_data, test_data = train, unif_train, validation, test

In [15]:
train_dense = train_data.to_dense()

In [16]:
train_dense.shape

torch.Size([15400, 1000])

In [17]:
users_unif, items_unif = unif_train_data._indices()[0], unif_train_data._indices()[1]
y_unif = unif_train_data._values()

In [18]:
train_loader = utils.data_loader.Block(train_data, u_batch_size=training_args['block_batch'][0], i_batch_size=training_args['block_batch'][1], device=device)
val_loader = utils.data_loader.DataLoader(utils.data_loader.Interactions(val_data), batch_size=training_args['batch_size'], shuffle=False, num_workers=0)
test_loader = utils.data_loader.DataLoader(utils.data_loader.Interactions(test_data), batch_size=training_args['batch_size'], shuffle=False, num_workers=0)

In [19]:
type(train_loader)

utils.data_loader.Block

In [34]:
class ThreeLinear(nn.Module):
  def __init__(self, n_user, n_item, n):
    super().__init__()
    
    self.user_bias = nn.Embedding(n_user, 1)
    self.item_bias = nn.Embedding(n_item, 1)
    self.data_bias = nn.Embedding(n, 1)
    self.init_embedding(0)
  
  def init_embedding(self, init):
    nn.init.kaiming_normal_(self.user_bias.weight, mode='fan_out', a = init)
    nn.init.kaiming_normal_(self.item_bias.weight, mode='fan_out', a = init)
    nn.init.kaiming_normal_(self.data_bias.weight, mode='fan_out', a = init)
    self.data_bias.weight.data *= 0.001
    
  def forward(self, users, items, values):
    u_bias = self.user_bias(users)
    i_bias = self.user_bias(items)
    d_bias = self.user_bias(users)
    
    preds = u_bias + i_bias + d_bias
    return preds.squeeze()
    

In [35]:
# data shape
n_user, n_item = train_data.shape

# Base model and its optimizer. This optimizer is for optimize parameters in base model using the updated weights (true optimization).
base_model = MetaMF(n_user, n_item, dim=base_model_args['emb_dim'], dropout=0).to(device)
base_optimizer = torch.optim.SGD(base_model.params(), lr=base_model_args['learning_rate'], weight_decay=0) # todo: other optimizer SGD

# Weight model and its optimizer. This optimizer is for optimize parameters of weight model. 
weight1_model = ThreeLinear(n_user, n_item, 2).to(device)
weight1_optimizer = torch.optim.Adam(weight1_model.parameters(), lr=weight1_model_args['learning_rate'], weight_decay=weight1_model_args['weight_decay'])

weight2_model = ThreeLinear(n_user, n_item, 2).to(device)
weight2_optimizer = torch.optim.Adam(weight2_model.parameters(), lr=weight2_model_args['learning_rate'], weight_decay=weight2_model_args['weight_decay'])

imputation_model = OneLinear(3).to(device)
imputation_optimizer = torch.optim.Adam(imputation_model.parameters(), lr=imputation_model_args['learning_rate'], weight_decay=imputation_model_args['weight_decay'])


In [36]:
# loss_criterion
sum_criterion = nn.MSELoss(reduction='sum')
none_criterion = nn.MSELoss(reduction='none')

In [79]:
stable_weight1_model = ThreeLinear(n_user, n_item, 2).to(device)
stable_weight1_model.load_state_dict(torch.load("/home/dable/Robust_Deconfounder_master/datasets/yahooR3/propensity_auto/ps1.pth.tar"))

stable_weight2_model = ThreeLinear(n_user, n_item, 2).to(device)
stable_weight2_model.load_state_dict(torch.load("/home/dable/Robust_Deconfounder_master/datasets/yahooR3/propensity_auto/ps2.pth.tar"))

<All keys matched successfully>

### Train start

In [37]:
training_loss = 0

In [38]:
n_user, n_item = train_data.shape
print(n_user, n_item)

15400 1000


In [39]:
# Early stopping
stopping_args = Stop_args(patience=training_args['patience'], max_epochs=training_args['epochs'])
early_stopping = EarlyStopping(base_model, **stopping_args)

In [40]:
users, items = next(iter(train_loader.User_loader)), next(iter(train_loader.Item_loader))

In [41]:
print(users) # dim: 6000

tensor([ 9851, 11046,   293,  ..., 13523, 13343, 13636], device='cuda:0')


In [42]:
print(items) # dim: 500

tensor([ 31, 681, 191, 200, 775, 452, 300, 765, 696, 391,  86, 939, 937, 338,
        684, 420, 166, 256, 852, 566, 768, 364, 604, 165, 354, 506, 316, 683,
        509, 365, 938, 679, 588, 219, 470, 413, 374, 203, 812, 648, 999, 969,
          7, 560, 260, 883, 501, 124, 623,  34, 798, 175, 211, 868, 671, 369,
        954, 282,  77, 857,  45, 632, 732, 608, 906, 870, 572, 327, 372, 533,
        873,  15, 138, 784, 901, 209, 199, 451, 541, 698,  64, 188,   2, 948,
        690, 845, 886,  65, 263, 457, 824, 719, 110, 607,  99,  26, 776, 685,
        844, 466, 854, 539, 907,  28, 252, 336, 729, 489, 201, 990, 895, 135,
        232, 395, 488, 301, 302, 916, 450, 445, 748, 284, 581,  49, 638, 527,
        159, 215, 216, 774, 903, 304, 773, 342,  52, 524, 749,  90, 556, 483,
        922, 752, 225,  83, 244, 208, 833, 292, 315, 904, 237, 864,  13, 704,
        473, 449, 677, 672, 619, 924, 498, 927, 117, 610, 703,  98, 436, 808,
         56, 928, 478, 348, 971, 701, 715, 184, 194, 855,   0, 1

In [43]:
# y_train: all 1's
# dim of users_train, items_train ~ 25000 (train._nnz() * 6000 * 500)
users_train, items_train, y_train = train_loader.get_batch(users, items, device)

In [44]:
users_train.size(), items_train.size(), y_train.size()

(torch.Size([24661]), torch.Size([24661]), torch.Size([24661]))

### Step 1: Assumed update of $\theta$ (update parameters one_step )

In [47]:
# calculate weight 1: w_k^{(1)}, only for training data (Eq 17.)
weight1_model.train()
weight1 = weight1_model(users_train, items_train, (y_train==1)*1)
weight2 = torch.exp(torch.nn.ReLU()(weight1/10)) # for stable training

In [48]:
# all pair
all_pair = torch.cartesian_prod(users, items)
users_all, items_all = all_pair[:,0], all_pair[:,1]

In [49]:
# calculate weight2: w_{ui}^{(2)} (Eq 17.)
# ((train_dense[users_all, items_all]!=0)*1): unobserved -> 1, observed -> 0
weight2_model.train()
weight2 = weight2_model(users_all, items_all, ((train_dense[users_all, items_all]!=0)*1))
weight2 = torch.exp(torch.nn.ReLU()(weight2/10)) # for stable training

In [50]:
# calculate imputation values
imputation_model.train()
impu_all = torch.tanh(imputation_model((train_dense[users_all, items_all]).long()))

### 1. Assumed Update of theta (Black arrow)

In [51]:
# one_step_model : assumed model, just update one step on base model. It is for updating parameters
one_step_model = MetaMF(n_user, n_item, dim=base_model_args['emb_dim'], dropout=0)

In [52]:
one_step_model.load_state_dict(base_model.state_dict())

one_step_model.train()


MetaMF(
  (user_latent): MetaEmbed()
  (item_latent): MetaEmbed()
  (user_bias): MetaEmbed()
  (item_bias): MetaEmbed()
  (dropout): Dropout(p=0, inplace=False)
)

In [53]:
# formal parameter: using training set to update parameters
# 1) observation data
y_hat_obs = one_step_model(users_train, items_train)
cost_obs = none_criterion(y_hat_obs, y_train)
loss_obs = torch.sum(weight1 * cost_obs)

In [54]:
# 2) all pair data 
y_hat_all = one_step_model(users_all, items_all)
cost_all = none_criterion(y_hat_all, impu_all)
loss_all = torch.sum(weight2 * cost_all)

In [55]:
# loss: \hat{L}_{T}(f | \phi)
loss = loss_obs + base_model_args['imputaion_lambda'] * loss_all + base_model_args['weight_decay'] * one_step_model.l2_norm(users_all, items_all)

In [56]:
# update parameters of one_step_model
one_step_model.zero_grad()
grads = torch.autograd.grad(loss, (one_step_model.params()), create_graph=True)
one_step_model.update_params(base_model_args['learning_rate'], source_params=grads)

### Step 2. Update of $\phi(\psi)$

In [57]:
# latter hyper_parameter: using uniform set to update hyper_parameters
y_hat_l = one_step_model(users_unif, items_unif)
loss_l = sum_criterion(y_hat_l, y_unif)

In [58]:
# update hyper-parameters
weight1_optimizer.zero_grad()
weight2_optimizer.zero_grad()
imputation_optimizer.zero_grad()

loss_l.backward()

weight1_optimizer.step()
weight2_optimizer.step()
imputation_optimizer.step()

### Adversarial Training

In [59]:
weight1_model_ad = ThreeLinear(n_user, n_item, 2).to(device)
weight2_model_ad = ThreeLinear(n_user, n_item, 2).to(device)
imputation_model_ad = OneLinear(3).to(device)

In [61]:
# load the weights of coresponding non-adversarial model
weight1_model_ad.load_state_dict(weight1_model.state_dict())
weight2_model_ad.load_state_dict(weight2_model.state_dict())
imputation_model_ad.load_state_dict(imputation_model.state_dict())

<All keys matched successfully>

In [62]:
weight1_model_ad.train()
weight1_ad = weight1_model_ad(users_train, items_train, (y_train==1)*2)
weight1_ad = torch.exp(torch.nn.ReLU()(weight1_ad/10))

In [63]:
weight2_model_ad.train()
weight2_ad = weight2_model_ad(users_all, items_all,(train_dense[users_all,items_all]!=0)*1)
weight2_ad = torch.exp(torch.nn.ReLU()(weight2_ad/10))

In [64]:
imputation_model_ad.train()
impu_ad_all = torch.tanh(imputation_model_ad((train_dense[users_all,items_all]).long()+1))

In [65]:
one_step_model_ad = MetaMF(n_user, n_item, dim=base_model_args['emb_dim'], dropout=0)
one_step_model_ad.load_state_dict(base_model.state_dict())

<All keys matched successfully>

In [66]:
one_step_model_ad.train()

# observation data
y_hat_ad_obs = one_step_model_ad(users_train, items_train)
cost_ad_obs = none_criterion(y_hat_ad_obs, y_train)
loss_ad_obs = torch.sum(cost_ad_obs * weight1_ad)

# all pair data in this block
y_hat_ad_all = one_step_model_ad(users_all, items_all)
cost_ad_all = none_criterion(y_hat_ad_all, impu_ad_all)
loss_ad_all = torch.sum(cost_ad_all * weight2_ad)

loss_ad = loss_ad_obs + base_model_args['imputaion_lambda'] * loss_ad_all + base_model_args['weight_decay'] * one_step_model_ad.l2_norm(users_all, items_all)

In [67]:
# update parameters of one_step_model
one_step_model_ad.zero_grad()
grads = torch.autograd.grad(loss_ad, (one_step_model_ad.params()), create_graph=True)
one_step_model_ad.update_params(base_model_args['learning_rate'], source_params=grads)

In [68]:
# adversarial learning on biased data
y_hat_ad = one_step_model_ad(users_train, items_train)
loss_ad = -1 * sum_criterion(y_hat_ad, y_train) # -1: min -f = max f

In [69]:
loss_ad

tensor(-28298.2051, device='cuda:0', grad_fn=<MulBackward0>)

In [71]:
# adversal update hyper-parameters
weight1_ad_optimizer = torch.optim.SGD(weight1_model_ad.parameters(), lr=0.1, weight_decay=0)
weight2_ad_optimizer = torch.optim.SGD(weight2_model_ad.parameters(), lr=0.1, weight_decay=0)
weight1_ad_optimizer.zero_grad()
weight2_ad_optimizer.zero_grad()


In [72]:
loss_ad.backward()

In [73]:
weight1_ad_optimizer.step()
weight2_ad_optimizer.step()

In [74]:
weight1_after_adv = weight1_model_ad(users_train, items_train,(y_train==1)*1)
weight1_after_adv = torch.exp(torch.nn.ReLU()(weight1_after_adv / 10))

weight2_after_adv = weight2_model_ad(users_all, items_all,(train_dense[users_all,items_all]!=0)*1)
weight2_after_adv = torch.exp(torch.nn.ReLU()(weight2_after_adv / 10))

In [88]:
def inner_bound(Gamma_Auto, ips_adv, ips_hat):
  upbound = torch.ones_like(ips_hat) + (ips_hat - torch.ones_like(ips_hat)) / Gamma_Auto
  lowbound = torch.ones_like(ips_hat) + (ips_hat - torch.ones_like(ips_hat)) * Gamma_Auto
  
  print(f'lowbound: {lowbound}')
  print(f'upbound: {upbound}')
  print(f'comparison: {upbound > lowbound}')
  
  # over_up_mask: upbound 보다 크면 1, 아니면 0
  # over_low_mask: lowbound 보다 작으면 1, 아니면 0
  over_up_mask = torch.where(ips_adv>upbound, torch.ones_like(ips_hat), torch.zeros_like(ips_hat))
  over_low_mask = torch.where(ips_adv<lowbound, torch.ones_like(ips_hat), torch.zeros_like(ips_hat))
  
  print(f'over_up_mask: {over_up_mask}')
  print(f'over_low_mask: {over_low_mask}')
  
  # torch.ones_like(ips_hat) - over_up_mask - over_low_mask: if two masking conditions are not satisfied, set 1 which means we will use ips_adv
  return over_up_mask*upbound + over_low_mask*lowbound + (torch.ones_like(ips_hat) - over_up_mask - over_low_mask) * ips_adv

In [89]:
# inner bound
ancor_1 = stable_weight1_model(users_train, items_train, (y_train==1)*1)
ancor_1 = torch.exp(torch.nn.ReLU()(ancor_1 / 10))
ancor_2 = stable_weight2_model(users_all, items_all, (train_dense[users_all,items_all]!=0)*1)
ancor_2 = torch.exp(torch.nn.ReLU()(ancor_2 / 10))

weight1 = inner_bound(3, weight1_after_adv, ancor_1)
weight2 = inner_bound(3, weight2_after_adv, ancor_2)

lowbound: tensor([1.0809, 1.0000, 1.0000,  ..., 1.0128, 1.0128, 1.0000], device='cuda:0',
       grad_fn=<AddBackward0>)
upbound: tensor([1.0090, 1.0000, 1.0000,  ..., 1.0014, 1.0014, 1.0000], device='cuda:0',
       grad_fn=<AddBackward0>)
comparison: tensor([False, False, False,  ..., False, False, False], device='cuda:0')
over_up_mask: tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0')
over_low_mask: tensor([1., 0., 0.,  ..., 1., 1., 0.], device='cuda:0')
lowbound: tensor([1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000], device='cuda:0',
       grad_fn=<AddBackward0>)
upbound: tensor([1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000], device='cuda:0',
       grad_fn=<AddBackward0>)
comparison: tensor([False, False, False,  ..., False, False, False], device='cuda:0')
over_up_mask: tensor([1., 0., 1.,  ..., 1., 1., 0.], device='cuda:0')
over_low_mask: tensor([0., 1., 0.,  ..., 0., 0., 1.], device='cuda:0')


In [ ]:
impu_all = torch.tanh(imputation_model((train_dense[users_all,items_all]).long()+1))

### Step 3. Update of $\theta$

In [ ]:
# loss of training set
base_model.train()
# all pair
y_hat_all = base_model(users_all, items_all)
cost_all = none_criterion(y_hat_all, impu_all)
loss_all = torch.sum(cost_all * weight2)
# observation
y_hat_obs = base_model(users_train, items_train)
cost_obs = none_criterion(y_hat_obs, y_train)
loss_obs = torch.sum(cost_obs * weight1)
loss = loss_obs + base_model_args['imputaion_lambda'] * loss_all + base_model_args['weight_decay'] * base_model.l2_norm(users_all, items_all)

base_optimizer.zero_grad()
loss.backward()
base_optimizer.step()

### Validation

In [69]:
base_model.eval()
with torch.no_grad():
    # training metrics
    train_pre_ratings = torch.empty(0).to(device)
    train_ratings = torch.empty(0).to(device)
    for u_batch_idx, users in enumerate(train_loader.User_loader): 
        for i_batch_idx, items in enumerate(train_loader.Item_loader): 
            users_train, items_train, y_train = train_loader.get_batch(users, items, device)
            pre_ratings = base_model(users_train, items_train)
            train_pre_ratings = torch.cat((train_pre_ratings, pre_ratings))
            train_ratings = torch.cat((train_ratings, y_train))

    # validation metrics
    val_pre_ratings = torch.empty(0).to(device)
    val_ratings = torch.empty(0).to(device)
    for batch_idx, (users, items, ratings) in enumerate(val_loader):
        pre_ratings = base_model(users, items)
        val_pre_ratings = torch.cat((val_pre_ratings, pre_ratings))
        val_ratings = torch.cat((val_ratings, ratings))

train_results = utils.metrics.evaluate(train_pre_ratings, train_ratings, ['MSE'], device)
val_results = utils.metrics.evaluate(val_pre_ratings, val_ratings, ['MSE', 'NLL', 'AUC'], device)


### Full training code

In [ ]:
for epoch in range(early_stopping.max_epochs):
    for u_batch_idx, users in enumerate(train_loader.User_loader):
        for i_batch_idx, items in enumerate(train_loader.Item_loader):
            # observation data in this batch
            users_train, items_train, y_train = train_loader.get_batch(users, items, device)

## Train & Eval

In [48]:
# train_and_eval(train, 
#               unif_train,
#               validation,
#               test,
#               device, 
#               base_model_args=base_model_args, 
#               weight1_model_args=weight1_model_args, 
#               weight2_model_args=weight2_model_args, 
#               imputation_model_args=imputation_model_args, 
#               training_args=training_args)

In [49]:
# transform sparse to dense matrix
train_data = train
train_dense = train_data.to_dense()
train_dense

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [50]:
unif_train_data = unif_train
users_unif = unif_train_data._indices()[0]
items_unif = unif_train_data._indices()[1]
y_unif = unif_train_data._values()

In [51]:
# build data_loader. (block matrix data loader)

train_loader = utils.data_loader.Block(train_data,
                                       u_batch_size=training_args['block_batch'][0],
                                       i_batch_size=training_args['block_batch'][1],
                                       device=device)

In [52]:
val_data, test_data = validation, test
val_loader = utils.data_loader.DataLoader(utils.data_loader.Interactions(val_data), batch_size=training_args['batch_size'], shuffle=False, num_workers=0)
test_loader = utils.data_loader.DataLoader(utils.data_loader.Interactions(test_data), batch_size=training_args['batch_size'], shuffle=False, num_workers=0)

In [53]:
# data shape
n_user, n_item = train_data.shape
print(n_user, n_item)

15400 1000


## Models

In [54]:
class MF(nn.Module):
  """
  Base module for matrix factoriazation
  """
  def __init__(self, n_user, n_item, dim=40, dropout=0, init=None):
    super().__init__()
    
    self.user_latent = nn.Embedding(n_user, dim)
    self.item_latent = nn.Embedding(n_item, dim)
    
    self.user_bias = nn.Embedding(n_user, 1)
    self.item_bias = nn.Embedding(n_item, 1)
    
    self.dropout_p = dropout
    self.dropout = nn.Dropout(p=self.dropout_p)
    if not init:
      self.init_embedding(init)
    else:
      self.init_embedding(0)
    
  def init_embedding(self, init):
    nn.init.kaiming_normal_(self.user_latent.weight.data, mode='fan_out', a=init)
    nn.init.kaiming_normal_(self.item_latent.weight.data, mode='fan_out', a=init)
    nn.init.kaiming_normal_(self.user_bias.weight.data, mode='fan_out', a=init)
    nn.init.kaiming_normal_(self.item_bias.weight.data, mode='fan_out', a=init)
    
  def forward(self, users, items):
    u_latent = self.dropout(self.user_latent(users))
    i_latent = self.dropout(self.item_latent(items))
    
    u_bias = self.user_bias(users)
    i_bias = self.user_bias(items)
    
    preds = torch.sum(u_latent * i_latent, dim=1, keepdim=True) + u_bias + i_bias
    
    return preds.squeeze(dim=-1)
  
  def l2_norm(self, users, items):
    users = torch.unique(users)
    items = torch.unique(items)
    
    l2_loss = (torch.sum(self.user_latent(users)**2) + torch.sum(self.item_latent(items)**2)) / 2
    return l2_loss


In [55]:
from model import MetaModule, MetaEmbed
class MetaEmbed(MetaModule):
  """
  Base module for matrix factorization
  """
  def __init__(self, dim_1, dim_2):
    super().__init__()
    ignore = nn.Embedding(dim_1, dim_2)
    
    self.register_buffer('weight', to_var(ignore.weight.data, requires_grad=True))
    self.register_buffer('bias', None)
    
  def forward(self):
    return self.weight
  
  def named_leaves(self):
    return [('weight', self.weight), ('bias', self.bias)]

In [56]:
class MetaMF(MetaModule):
    """
    Base module for matrix factorization.
    """
    def __init__(self, n_user, n_item, dim=40, dropout=0, init = None):
        super().__init__()
        
        self.user_latent = MetaEmbed(n_user, dim)
        self.item_latent = MetaEmbed(n_item, dim)
        self.user_bias = MetaEmbed(n_user, 1)
        self.item_bias = MetaEmbed(n_item, 1)
        self.dropout_p = dropout
        self.dropout = nn.Dropout(p=self.dropout_p)
        if init is not None:
            self.init_embedding(init)
        else: 
            self.init_embedding(0)
        
    def init_embedding(self, init): 

        nn.init.kaiming_normal_(self.user_latent.weight, mode='fan_out', a = init)
        nn.init.kaiming_normal_(self.item_latent.weight, mode='fan_out', a = init)
        nn.init.kaiming_normal_(self.user_bias.weight, mode='fan_out', a = init)
        nn.init.kaiming_normal_(self.item_bias.weight, mode='fan_out', a = init)
          
    def forward(self, users, items):
        u_latent = self.dropout(self.user_latent.weight[users])
        i_latent = self.dropout(self.item_latent.weight[items])
        u_bias = self.user_bias.weight[users]
        i_bias = self.item_bias.weight[items]

        preds = torch.sum(u_latent * i_latent, dim=1, keepdim=True) + u_bias + i_bias
        return preds.squeeze(dim=-1)

    def l2_norm(self, users, items, unique = True): 

        users = torch.unique(users)
        items = torch.unique(items)
        
        l2_loss = (torch.sum(self.user_latent.weight[users]**2) + torch.sum(self.item_latent.weight[items]**2)) / 2
        return l2_loss

In [57]:
# import torch
# import torch.nn as nn

# # Define the embedding layer
# embedding_dim = 10
# vocab_size = 100
# embedding_layer = nn.Embedding(vocab_size, embedding_dim)

# # Example input indices
# indices = torch.tensor(list(range(4)))

# # Apply the embedding layer
# embedded_output = embedding_layer(indices)

# print(embedded_output.shape)

In [58]:
base_model = MetaMF(n_user, n_item, dim=base_model_args['emb_dim'], dropout=0).to(device)
base_optimizer = torch.optim.SGD(base_model.params(), lr=base_model_args['learning_rate'], weight_decay=0) # todo: other optimizer SGD

register_buffer 로 layer를 등록하면 어떤 특징이 있는가?

1. optimizer가 업데이트하지 않는다.

2. 그러나 값은 존재한다(하나의 layer로써 작용한다고 보면 된다.)

3. state_dict()로 확인이 가능하다.

4. GPU연산이 가능하다.

 

따라서 네트워크를 구성함에 있어서 네트워크를 end2end로 학습시키고 싶은데 중간에 업데이트를 하지않는 일반 layer를 넣고 싶을 때 사용할 수 있다.

In [59]:
def to_var(x, requires_grad=True):
  x = x.cuda() if torch.cuda.is_available() else x
  return Variable(x, requires_grad=requires_grad) # 현재는 모든 tensor에서 required_grad 옵션을 통해 gradient를 추적할 수 있기 때문에 따로 위와 같이 Variable로 감싸줄 필요가 없다. (현재는 쓸 필요 없음)


In [60]:
class OneLinear(nn.Module):
  """
  linear model: r
  """
  def __init__(self, n):
    super().__init__()

    self.bias = nn.Embedding(n,1)
    self.init_embedding()

  def init_embedding(self):
    self.bias.weight.data *= 0.01

  def forward(self, values):
    d_bias = self.bias(values)
    return d_bias.squeeze()


In [61]:
class TwoLinear(nn.Module):
  def __init__(self, n_user, n_item):
    super().__init__()

    self.user_bias = nn.Embedding(n_user, 1)
    self.item_bias = nn.Embedding(n_item, 1)

    self.init_embedding(0)
  
  def init_embedding(self, init):
    nn.init.kaiming_normal_(self.user_bias.weight, mode='fan_out', a=init)
    nn.init.kaiming_normal_(self.item_bias.weight, mode='fan_out', a=init)

  def forward(self, users, items):
    u_bias = self.user_bias(users)
    i_bias = self.item_bias(items)
    preds = u_bias + i_bias
    
    return preds.squeeze()

In [62]:
class ThreeLinear(nn.Module):
  """
  linear model: u + i + r / o
  """
  def __init__(self, n_user, n_item, n):
    super().__init__()
    
    self.user_bias = nn.Embedding(n_user, 1)
    self.item_bias = nn.Embedding(n_item, 1)
    self.data_bias= nn.Embedding(n, 1)
    self.init_embedding(0)
      
  def init_embedding(self, init): 
    nn.init.kaiming_normal_(self.user_bias.weight, mode='fan_out', a = init)
    nn.init.kaiming_normal_(self.item_bias.weight, mode='fan_out', a = init)
    nn.init.kaiming_normal_(self.data_bias.weight, mode='fan_out', a = init)
    self.data_bias.weight.data *= 0.001

  def forward(self, users, items, values):

    u_bias = self.user_bias(users)
    i_bias = self.item_bias(items)
    d_bias = self.data_bias(values)

    preds = u_bias + i_bias + d_bias
    return preds.squeeze()

In [63]:
weight1_model = TwoLinear(n_user, n_item).to(device)
weight1_optimizer = torch.optim.Adam(weight1_model.parameters(), 
                                     lr=base_model_args['learning_rate'],
                                     weight_decay=0)

In [64]:
weight2_model = ThreeLinear(n_user, n_item, 2).to(device)
weight2_optimizer = torch.optim.Adam(weight2_model.parameters(), 
                                     lr=weight2_model_args['learning_rate'],
                                     weight_decay=weight2_model_args['weight_decay'])

In [65]:
imputation_model = OneLinear(2).to(device)
imputation_optimizer = torch.optim.Adam(imputation_model.parameters(),
                                        lr=imputation_model_args['learning_rate'],
                                        weight_decay=imputation_model_args['weight_decay'])

In [66]:
# loss criterion
sum_criterion = nn.MSELoss(reduction='sum')
none_criterion = nn.MSELoss(reduction='none')

## Training

In [67]:
train_dense.shape

torch.Size([15400, 1000])

In [68]:
stopping_args = Stop_args(patience=training_args['patience'], max_epochs=training_args['epochs'])
early_stopping = EarlyStopping(base_model, **stopping_args)

In [76]:
from train_implicit import train_and_eval

# setting that works well
train_and_eval(train, 
               unif_train, 
               validation, 
               test,
               device,
               base_model_args = base_model_args, 
               weight1_model_args = weight1_model_args,
               weight2_model_args = weight2_model_args,
               imputation_model_args = imputation_model_args, 
               training_args = training_args)